In [1]:
from data_loader import get_north_atlantic_sst_data, get_antarctic_sea_ice_extent_data


2023-06-26 19:58:10.459 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2023-06-26 19:58:10.461 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [11]:
sie_df = get_antarctic_sea_ice_extent_data()

In [12]:
sie_df.head()

,Year,Month,Day,Extent,Missing,Source Data
0,1978,10,26,17.624,0.0,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.26/nt_197...
1,1978,10,28,17.803,0.0,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.28/nt_197...
2,1978,10,30,17.670,0.0,['/ecs/DP1/PM/NSIDC-0051.001/1978.10.30/nt_197...
3,1978,11,1,17.527,0.0,['/ecs/DP1/PM/NSIDC-0051.001/1978.11.01/nt_197...
4,1978,11,3,17.486,0.0,['/ecs/DP1/PM/NSIDC-0051.001/1978.11.03/nt_197...


In [13]:
sie_df = sie_df.drop(columns=['Source Data', 'Missing'])

In [14]:
sie_df.head()

,Year,Month,Day,Extent
0,1978,10,26,17.624
1,1978,10,28,17.803
2,1978,10,30,17.670
3,1978,11,1,17.527
4,1978,11,3,17.486


In [32]:
import pandas as pd
import plotly.graph_objs as go
import numpy as np
from datetime import datetime

from data_loader import get_north_atlantic_sst_data, get_antarctic_sea_ice_extent_data
from matplotlib import cm
import matplotlib

def sie_anomaly_fig():
    df = get_antarctic_sea_ice_extent_data()
    df = df.drop(columns=['Source Data', 'Missing'])

    # Convert 'Year', 'Month', 'Day' to a datetime format
    df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

    # Create a new column for the formatted date string
    df['date_str'] = df['date'].dt.strftime('%d-%m-%Y')

    # Convert date to day of the year
    df['day_of_year'] = df['date'].dt.dayofyear

    # Calculate anomaly (subtract 1982-2011 mean)
    day_of_year_mean_1982_2011 = df[df['date'].dt.year.isin(range(1982, 2012))].groupby(df['date'].dt.dayofyear)['Extent'].mean()
    day_of_year_mean_1982_2011 = day_of_year_mean_1982_2011.reset_index()

    day_of_year_mean_1982_2011.columns = ['day_of_year', 'mean']
    df = df.merge(day_of_year_mean_1982_2011, on='day_of_year', suffixes=('', '_mean'))
    
    df['Extent'] = df['Extent'] - df['mean']
#     df = df.sort_values('day_of_year')

    # Get the unique years
    years = df['Year'].unique()

    # Get the current year
    current_year = datetime.now().year

    # Get a color map
    cmap = cm.get_cmap('plasma', len(years))

    # Initialize the figure
    fig = go.Figure()

    # Add a line for each year
    for i, year in enumerate(years):
        color = matplotlib.colors.rgb2hex(cmap(i)[:3])
        year_data = df[df['Year'] == year]
        if year == current_year:
            fig.add_trace(go.Scatter(x=year_data['day_of_year'], y=year_data['Extent'],
                                    mode='lines', name=str(year), line=dict(color='red', width=3),
                                    hovertemplate=
                                    '<b>Date</b>: %{customdata}<br>' +
                                    '<b>Extent</b>: %{y:.2f}<br>'
                                    '<extra></extra>',
                                    customdata=year_data['date_str']))
        else:
            fig.add_trace(go.Scatter(x=year_data['day_of_year'], y=year_data['Extent'],
                                     mode='lines', name=str(year), line=dict(color=color), opacity=0.3,
                                     hovertemplate=
                                     '<b>Date</b>: %{customdata}<br>' +
                                     '<b>Extent</b>: %{y:.2f}<br>'
                                     '<extra></extra>',
                                     customdata=year_data['date_str']))

    # Set layout properties
    fig.update_layout(title='Antarctic Sea Ice Extent Anomaly', xaxis_title='Day of Year',
                      yaxis_title='Ice Extent difference from 1982-2011 mean (10^6 sq km)',
                      width=900, height=600, legend={'traceorder': 'reversed'})
    return fig, df

fig, df = sie_anomaly_fig()
fig

/var/folders/vp/hdjztmdd2q92v8q658y04fbw0000gn/T/ipykernel_56725/715637483.py:40: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



In [27]:
df.sort_values('day_of_year')

,Year,Month,Day,Extent,date,date_str,day_of_year,mean,Extent_diff
8727,1980,1,1,5.967,1980-01-01,01-01-1980,1,7.019080,-1.052080
8750,2008,1,1,9.120,2008-01-01,01-01-2008,1,7.019080,2.100920
8751,2009,1,1,8.007,2009-01-01,01-01-2009,1,7.019080,0.987920
8752,2010,1,1,7.456,2010-01-01,01-01-2010,1,7.019080,0.436920
8753,2011,1,1,6.843,2011-01-01,01-01-2011,1,7.019080,-0.176080
...,...,...,...,...,...,...,...,...,...
14655,1992,12,31,6.403,1992-12-31,31-12-1992,366,7.004714,-0.601714
14654,1988,12,31,7.261,1988-12-31,31-12-1988,366,7.004714,0.256286
14653,1984,12,31,7.171,1984-12-31,31-12-1984,366,7.004714,0.166286
14656,1996,12,31,6.334,1996-12-31,31-12-1996,366,7.004714,-0.670714


In [24]:
len(df['date'])

14663

In [30]:
def sie_anomaly_perc_fig():
    df = get_antarctic_sea_ice_extent_data()
    df = df.drop(columns=['Source Data', 'Missing'])

    # Convert 'Year', 'Month', 'Day' to a datetime format
    df['date'] = pd.to_datetime(df[['Year', 'Month', 'Day']])

    # Create a new column for the formatted date string
    df['date_str'] = df['date'].dt.strftime('%d-%m-%Y')

    # Convert date to day of the year
    df['day_of_year'] = df['date'].dt.dayofyear

    # Calculate anomaly (subtract 1982-2011 mean)
    day_of_year_mean_1982_2011 = df[df['date'].dt.year.isin(range(1982, 2012))].groupby(df['date'].dt.dayofyear)['Extent'].mean()
    day_of_year_mean_1982_2011 = day_of_year_mean_1982_2011.reset_index()

    day_of_year_mean_1982_2011.columns = ['day_of_year', 'mean']
    df = df.merge(day_of_year_mean_1982_2011, on='day_of_year', suffixes=('', '_mean'))

    df['Extent_diff_perc'] = (df['Extent'] - df['mean']) / df['mean'] * 100
    df = df.sort_values('day_of_year')

    # Get the unique years
    years = df['Year'].unique()

    # Get the current year
    current_year = datetime.now().year

    # Get a color map
    cmap = cm.get_cmap('plasma', len(years))

    # Initialize the figure
    fig = go.Figure()

    # Add a line for each year
    for i, year in enumerate(years):
        color = matplotlib.colors.rgb2hex(cmap(i)[:3])
        year_data = df[df['Year'] == year]
        if year == current_year:
            fig.add_trace(go.Scatter(x=year_data['day_of_year'], y=year_data['Extent_diff_perc'],
                                    mode='lines', name=str(year), line=dict(color='red', width=3),
                                    hovertemplate=
                                    '<b>Date</b>: %{customdata}<br>' +
                                    '<b>Extent diff</b>: %{y:.2f}%<br>'
                                    '<extra></extra>',
                                    customdata=year_data['date_str']))
        else:
            fig.add_trace(go.Scatter(x=year_data['day_of_year'], y=year_data['Extent_diff_perc'],
                                     mode='lines', name=str(year), line=dict(color=color), opacity=0.3,
                                     hovertemplate=
                                     '<b>Date</b>: %{customdata}<br>' +
                                     '<b>Extent diff</b>: %{y:.2f}%<br>'
                                     '<extra></extra>',
                                     customdata=year_data['date_str']))

    # Set layout properties
    fig.update_layout(title='Antarctic Sea Ice Extent Anomaly', xaxis_title='Day of Year',
                      yaxis_title='Ice Extent difference from 1982-2011 mean (10^6 sq km)',
                      width=900, height=600, legend={'traceorder': 'reversed'})
    return fig, df

fig, df = sie_anomaly_perc_fig()
fig

/var/folders/vp/hdjztmdd2q92v8q658y04fbw0000gn/T/ipykernel_56725/2859441726.py:31: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.

